<h1 align=center> Test the ML backend API with celery task queue</h1>


Okay, now it's getting serious. Start the model server. We can do this by sending a request to the API and checking if the response is what we expect.

In [ ]:
import requests
import asyncio
import sounddevice as sd
import time

import numpy as np

In [ ]:
HOST = 'http://localhost:5123/' # change this to your host

## Record an audio

In [ ]:
fs = 16000# 22050
duration = 5  # seconds

rec = sd.rec(fs*duration, samplerate=fs, channels=1, blocking=True, dtype='int16')

## Post a transcribe task / translate task to the task queue

In [ ]:
TASK = 'transcribe'
LANGUAGE = 'de'


In [ ]:
res = requests.post(HOST + f"transcribe/predict?task={TASK}" + (f"&language={LANGUAGE}" if LANGUAGE is not None
                     else ""), json={"data":rec.tolist()}).json()

try:
    task_id = res['task_id']
finally:
    print(res)

In [ ]:
while res["status"] != "Success" and res["status"] != "Failed":
    res = requests.get(HOST + "transcribe/result/" + (task_id)).json()
    print(res)
    time.sleep(2)

## Using the OpenAI API, translate text useing GPT3

In [ ]:
text = res['text']

source_language = res["language"]
text, source_language

In [ ]:
res = requests.post(HOST + f"translation/predict", json={"text":text, "language": source_language}).json()

try:
    task_id = res['task_id']
finally:
    print(res)

In [ ]:
while res["status"] != "Success" and res["status"] != "Failed":
    res = requests.get(HOST + "translation/result/" + (task_id))
    try:
        res = res.json()
        print(res)
    except:
        print(res.content)
        break
    time.sleep(2)
    


## Given some English text, synthesize and audio sequence

In [ ]:
text = res2["text"]

In [ ]:
res = requests.post(HOST + "tts/predict", json={"text":text}).json()

try:
    task_id = res['task_id']
finally:
    print(res)

In [ ]:
while res["status"] != "Success" and res["status"] != "Failed":
    res = requests.get(HOST + "tts/result/" + (task_id))
    try:
        res = res.json()
        print(res["status"])
    except:
        print(res.content)
        break
    time.sleep(2)

In [ ]:
sd.play(np.asarray(res["waveforms"]).reshape(-1,1), samplerate=22050)

### Cool, that works, now let's try a complete workflow:

In [ ]:
fs = 16000# For Whisper
duration = 6  # seconds

rec = sd.rec(fs*duration, samplerate=fs, channels=1, blocking=True, dtype='int16')

In [ ]:
res1 = requests.post(HOST + "transcribe/predict?task=transcribe", json={"data":rec.tolist()}).json()

while res1["status"] != "Success" and res1["status"] != "Failed":
    res1 = requests.get(HOST + "transcribe/result/" + (res1["task_id"])).json()
    time.sleep(1)
    
print(res1["text"])
res2 = requests.post(HOST + f"translation/predict", json={"text":res1["text"], "language": res1["language"]}).json()

while res2["status"] != "Success" and res2["status"] != "Failed":
    res2 = requests.get(HOST + "translation/result/" + (res2["task_id"])).json()
    time.sleep(1)

print(res2["text"])
res3 = requests.post(HOST + "tts/predict", json={"text":res2["text"]}).json()

while res3["status"] != "Success" and res3["status"] != "Failed":
    res3 = requests.get(HOST + "tts/result/" + (res3["task_id"])).json()
    time.sleep(1)



sd.play(np.asarray(res3["waveforms"]).reshape(-1,1), samplerate=22050)